# Sequential Graphs

## Introduction

This notebook demonstrates how to build sequential graphs for chatbot workflows using LangGraph and LangChain.

## Environment Setup

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
from rich import print

# Load environment variables from .env file
load_dotenv('../.env')

## Basic Chatbot Example

In [ ]:
llm = ChatOllama(model="llama3.2")
llm.invoke("Hello, world!").content

### Agent State Definition

In [ ]:
class AgentState(TypedDict):
    name: str
    age: str
    final: str
    message: str

### Node Functions

In [ ]:
def greating_message(state: AgentState) -> AgentState:
    """Function to greet the user and initialize the state."""
    print("Welcome to the chatbot!")
    state['name'] = input("What is your name? ")
    state['age'] = input("How old are you? ")
    return state

def chatbot(state: AgentState) -> AgentState:
    """Chatbot function that updates the state with user input."""
    user_input = input("You: ")
    state['message'] = llm.invoke(user_input).content
    print(f"Chatbot: {state['message']}")
    return state

---

## Sequential Graph with LangGraph and LangChain

In [ ]:
# simple_chatbot.py
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import ChatOpenAI

### 1. Define the Shared State Schema

In [ ]:
class State(TypedDict):
    name: str
    user_question: str
    messages: list[AnyMessage]

### 2. Node 1: Greet the User by Name

In [ ]:
def greet(state: State) -> dict:
    greeting = AIMessage(f"Hi {state['name']}!")
    # Append to existing messages
    return {"messages": state.get("messages", []) + [greeting]}

### 3. Node 2: Take the User's Question and Get an LLM Response

In [ ]:
def answer(state: State) -> dict:
    # Build the message history: include the user's question
    history = state["messages"] + [HumanMessage(state["user_question"])]
    # Call the chat model
    model = ChatOpenAI(model_name="gpt-3.5-turbo")
    response: AIMessage = model(history)
    return {"messages": history + [response]}

### 4. Assemble the Graph

In [ ]:
builder = StateGraph(State)
builder.add_node(greet)
builder.add_node(answer)
# Define control flow: START → greet → answer → END
builder.add_edge(START, "greet")
builder.add_edge("greet", "answer")
builder.add_edge("answer", END)
graph = builder.compile()

### 5. Invoke the Graph

In [ ]:
initial_state = {
    "name": "Alice",
    "user_question": "What's the capital of France?",
    "messages": []
}
result = graph.invoke(initial_state)

### 6. Print Out the Conversation

In [ ]:
for msg in result["messages"]:
    msg.pretty_print()